#### Explore the Data

In [22]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StructType as R, StructField as Fld,\
    DoubleType as Dbl, StringType as Str, IntegerType as Int,\
    TimestampType as Timestamp, DateType as Date, LongType as Long
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
import pandas as pd
import re
import configparser
import os
import shutil
from pathlib import Path
from datetime import datetime

In [23]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
config = configparser.ConfigParser()
config.read('etl.cfg')

os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']
AWS_ACCESS_KEY_ID = config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['AWS']['AWS_SECRET_ACCESS_KEY']


spark = SparkSession.builder\
        .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .config("spark.hadoop.fs.s3a.access.key",AWS_ACCESS_KEY_ID)\
        .config("spark.hadoop.fs.s3a.secret.key",AWS_SECRET_ACCESS_KEY)\
        .enableHiveSupport().getOrCreate()

In [24]:
# Procedure read out validation pair values from SAS Labels Description
def get_validation_code_from_SAS_labels(sas_input_label):
    '''
    This procedure read a input SAS Labels Description and then write out validation code datasets.
    The SAS Labels Description included validation code datasets with labels: I94RES (same to I94CIT), I94PORT, I94ADDR, I94MODE, I94VISA.
    
    Parameters
    ----------
    sas_input_label : string
        The label name of validation code dataset. Its can be one of I94RES (same to I94CIT), I94PORT, I94ADDR, I94MODE, I94VISA.
    
    Returns
    -------
    validation_code_list : validation_value_pairs(tuple(str_valid_code, str_valid_value))
        The return output is a specific SAS label list of validation code value pairs.
    '''

    # Read input SAS Labels Descriptions
    with open('I94_SAS_Labels_Descriptions.SAS') as sas_validation_code:
            labels_from_sas = sas_validation_code.read()

    # Parse labels from SAS Label Description input
    sas_labels = labels_from_sas[labels_from_sas.index(sas_input_label):]
    sas_labels = sas_labels[:sas_labels.index(';')]
    
    # Processing line by line, remove separate charaters and then append value pair
    lines = sas_labels.splitlines()
    validation_code_list = []
    for line in lines:
        try:
            valid_code, valid_value = line.split('=')
            valid_code = valid_code.strip().strip("'").strip('"')
            valid_value = valid_value.strip().strip("'").strip('"').strip()
            validation_code_list.append((valid_code, valid_value))
        except:
            pass
        
    return validation_code_list

In [25]:
# Procedure extract parts from SAS Labels Description
def extract_staging_sas_label(label):
    '''
    This procedure get a specific part of data dictionary from SAS_Labels_Descriptions.SAS to a schema.
    The dictionary part include valid_code and valid_value
    
    Parameters
    ----------
    label: 
        a string input of specific label from "SAS_Label_Descriptions.SAS"
        
    Syntax note: 
        input value in string datatype, need inside a pair of single quotes. Ex: 'I94RES', 'I94PORTS'
    
    Returns
    -------
    Dir of csv files with a specific part as input label.
    '''
    label_name = label
    valid_code = label + "_valid_code"
    valid_value = label + "_valid_value"
    csv_output = label + "_sas_label_validation"
    
    # Create dir for output
    parent_dir = "./"
    path = os.path.join(parent_dir, csv_output)
    try:
        os.makedirs(path, exist_ok = True)
        print("Directory '%s' created successfully" % csv_output)
    except OSError as error:
        print("Directory '%s' can not be created" % csv_output)

    # Define output dataframe structure
    schema = R([
        Fld(valid_code, Str()),
        Fld(valid_value, Str())
    ])

    # Create dataframe from extracted label
    df = spark.createDataFrame(
        data=get_validation_code_from_SAS_labels(label_name),
        schema=schema
    )

    # df.write.mode('overwrite').csv(csv_output)
    shutil.rmtree(csv_output, ignore_errors=False, onerror=None)
    df.write.options(header='True', delimiter=',').csv(csv_output)

    df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv(csv_output)

    print("Top 20 rows of {} ".format(csv_output))
    df.show()

    print("Count rows of {}: {} ".format(csv_output, df.count()))
    
    print("Check unique value of {}: {} ".format(csv_output, df.select(valid_code).distinct().count()))

    print("Staging csv files in: {}".format(csv_output))

    return df

In [26]:
# Procedure convert column name
def convert_column_names(df):
    '''
    This procedure standardizing column names to snake case format. Format ex: customer_name, billing_address, total_price.
    
    Parameters
    ----------
    dataframe : string_of_dataframe
        The input dataframe with column names might have elements of messy columns names, including accents, different delimiters, casing and multiple white spaces.
        Snake case style replaces the white spaces and symbol delimiters with underscore and converts all characters to lower case
    
    Returns
    -------
    Dataframe with column names has been changed to snake_case format.
    '''
    cols = df.columns
    column_name_changed = []

    for col in cols:
        new_column = col.lstrip().rstrip().lower().replace (" ", "_").replace ("-", "_")
        column_name_changed.append(new_column)

    df.columns = column_name_changed

In [27]:
# Procedure remove specific dir (if need)
def rmdir(directory):
    '''
    This procedure perform pure recursive a directory.
    
    Parameters
    ----------
    directory : string_of_path_to_dir
        The input directory is a path to target dir. This dir and all its belong child objects wil be deleted.
        Syntax note: rmdir(Path("target_path_to_dir"))
            with Path("target_path_to_dir") returns path to dir format as 'directory' input
    
    Returns
    -------
    None
    '''
    directory = Path(directory)
    for item in directory.iterdir():
        if item.is_dir():
            rmdir(item)
        else:
            item.unlink()
    directory.rmdir()

##### I94 Immigration dataset

In [10]:
# Read input dataset to Spark dataframe
# i94immi_dataset = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# i94_immi_df = spark.read.format('com.github.saurfang.sas.spark').load(i94immi_dataset)
i94immi_df = spark.read.parquet("sas_data")

In [ ]:
# Data type
i94immi_df.dtypes

In [ ]:
# Attributes columns
i94immi_df.show(5)

In [ ]:
i94immi_df.count()

In [14]:
# Create tableview for data manipulation
i94immi_df.createOrReplaceTempView('i94immi_table')

In [15]:
# Drop amount of un-makesance records cause by `DepartureDate >= ArrivalDate`
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE arrdate <= depdate
""").createOrReplaceTempView("i94immi_table")

In [16]:
# Add column `arrival_date = timestone + arrdate_offset_day`, with:
# - timestone = '1960-01-01' (***datetime*** datatype)
# - arrdate_offset_day = 'arrdate' (***integer*** datatype)
# - arrival_date (***datetime*** datatype)
spark.sql("""
    SELECT *, date_add(to_date('1960-01-01'), arrdate) AS arrival_date 
    FROM i94immi_table
""").createOrReplaceTempView("i94immi_table")

In [17]:
# Add column `departure_date = timestone + depdate_offset_day`, with:
# - `timestone` = '1960-01-01' (***datetime*** datatype)
# - `depdate_offset_day` = 'depdate' (***integer*** datatype)
# - `departure_date` (***datetime*** datatype)
spark.sql("""SELECT *, CASE 
                        WHEN depdate >= arrdate THEN date_add(to_date('1960-01-01'), depdate)
                        WHEN depdate IS NULL THEN NULL
                        ELSE 'NaN' END AS departure_date 
                FROM i94immi_table
            """).createOrReplaceTempView("i94immi_table")

In [18]:
# extracted i94mode from `I94_SAS_Labels_Descriptions_SAS`
# i94mode includes:
# {'1': 'Air', '2': 'Sea', '3': 'Land', '9': 'Not reported'}
# Keep air arrival only, mean keep `i94mode=1`
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE i94mode == 1.0
""").createOrReplaceTempView("i94immi_table")

In [19]:
# Mapping `i94visa` numbers to `visatype` instead
spark.sql("""
        SELECT *, CASE 
                    WHEN i94visa = 1.0 THEN 'Business' 
                    WHEN i94visa = 2.0 THEN 'Pleasure'
                    WHEN i94visa = 3.0 THEN 'Student'
                    ELSE 'NaN' END AS visa_type
        FROM i94immi_table
    """).createOrReplaceTempView("i94immi_table")

In [20]:
# Keep user records of `male = 'M'` and `female = 'F'` only
spark.sql("""
    SELECT * 
    FROM i94immi_table 
    WHERE gender IN ('F', 'M')
""").createOrReplaceTempView("i94immi_table")

In [21]:
# Drop NULL value on arrival state
spark.sql("""
    SELECT *
    FROM i94immi_table
    WHERE i94addr IS NOT NULL
""").createOrReplaceTempView("i94immi_table")

In [22]:
# Keep necessary columns
# Convert month and year timestamp
spark.sql("""
        SELECT 
            cicid,
            i94cit,
            i94res,
            i94port,
            arrival_date,
            YEAR(arrival_date) as i94yr,
            MONTH(arrival_date) as i94mon,
            i94mode,
            i94addr,
            departure_date,
            i94bir,
            i94visa,
            count,
            dtadfile,
            biryear,
            dtaddto,
            gender,
            insnum,
            airline,
            admnum,
            fltno,
            visatype,
            visa_type
        FROM i94immi_table
            """).createOrReplaceTempView('i94immi_table')

In [29]:
# Verify the cleaned
i94immi_df = spark.sql("""
    SELECT *
    FROM i94immi_table
""")

In [31]:
i94immi_df.show(3)

+---------+------+------+-------+------------+-----+------+-------+-------+--------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|    cicid|i94cit|i94res|i94port|arrival_date|i94yr|i94mon|i94mode|i94addr|departure_date|i94bir|i94visa|count|dtadfile|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|visa_type|
+---------+------+------+-------+------------+-----+------+-------+-------+--------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|5748517.0| 245.0| 438.0|    LOS|  2016-04-30| 2016|     4|    1.0|     CA|    2016-05-08|  40.0|    1.0|  1.0|20160430| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| Business|
|5748518.0| 245.0| 438.0|    LOS|  2016-04-30| 2016|     4|    1.0|     NV|    2016-05-17|  32.0|    1.0|  1.0|20160430| 1984.0|10292016|     F|  null|     VA|9.495562283E10|00007|      B1| Business|


In [124]:
# Verify the cleaned
spark.sql("""
    SELECT COUNT(*) as amount_i94immi_rows
    FROM i94immi_table
""").show()

+-------------------+
|amount_i94immi_rows|
+-------------------+
|            2377896|
+-------------------+



In [32]:
# Staging to csv file --> write a function write spark sql dataframe to csv
path = './i94immi_df_clean'
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % path)
except OSError as error:
    print("Directory '%s' can not be created" % path)

rmdir(Path("i94immi_df_clean")) # use this line from 2nd running
i94immi_df.write.options(header='True', delimiter=',').csv("i94immi_df_clean")

Directory './i94immi_df_clean' created successfully


##### Read out from staging csv to test S3 storage parquet files

In [28]:
# Verify out from staging csv partitions
i94immi_df = spark.read.options(inferSchema="true", delimiter=",", header = "true").csv("i94immi_df_clean")

In [29]:
# Prepare to save out s3 storage
i94immi_df.show(3)

+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|    cicid|i94cit|i94res|i94port|       arrival_date|i94yr|i94mon|i94mode|i94addr|     departure_date|i94bir|i94visa|count|dtadfile|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|visa_type|
+---------+------+------+-------+-------------------+-----+------+-------+-------+-------------------+------+-------+-----+--------+-------+--------+------+------+-------+--------------+-----+--------+---------+
|5748517.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     CA|2016-05-08 00:00:00|  40.0|    1.0|  1.0|20160430| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| Business|
|5748518.0| 245.0| 438.0|    LOS|2016-04-30 00:00:00| 2016|     4|    1.0|     NV|2016-05-17 00:00:00|  32.0|    1.0|  1.0|20160430| 1984.0|10292016|   

In [9]:
i94immi_df.dtypes

[('cicid', 'double'),
 ('i94cit', 'double'),
 ('i94res', 'double'),
 ('i94port', 'string'),
 ('arrival_date', 'timestamp'),
 ('i94yr', 'int'),
 ('i94mon', 'int'),
 ('i94mode', 'double'),
 ('i94addr', 'string'),
 ('departure_date', 'timestamp'),
 ('i94bir', 'double'),
 ('i94visa', 'double'),
 ('count', 'double'),
 ('dtadfile', 'int'),
 ('biryear', 'double'),
 ('dtaddto', 'string'),
 ('gender', 'string'),
 ('insnum', 'string'),
 ('airline', 'string'),
 ('admnum', 'double'),
 ('fltno', 'string'),
 ('visatype', 'string'),
 ('visa_type', 'string')]

In [30]:
# Change to Spark SQl
i94immi_df.createOrReplaceTempView('fact_i94immi')

In [31]:
# Create fact table
fact_i94immi = spark.sql("""
        SELECT
            cicid as travel_cicid,
            i94cit as from_country_code,
            i94res as immi_country_code,
            i94port as arrival_port_code,
            arrival_date as immi_arrival_date,
            i94yr as arrival_year,
            i94mon as arrival_month,
            i94mode as airline_mode_code,
            i94addr as immi_state_code,
            departure_date,
            i94bir as traveller_age,
            i94visa as visatype_by_number,
            biryear as traveller_birth_year,
            gender as traveller_sex,
            fltno as immi_flight_code,
            visatype as visatype_by_code,
            visa_type
        FROM fact_i94immi
    """)

In [32]:
# S3 URI `s3://capstone-project-outputs/s3_outputs/`
# Copy S3 URI = 's3a://dendcapstonetest/s3_outputs/'
# update this to your bucket name
s3_uri = 's3://capstone-project-outputs/s3_outputs/'
parquet_outputs = s3_uri + 'unitTest_fact_i94immi.parquet'
parquet_outputs

's3://capstone-project-outputs/s3_outputs/unitTest_fact_i94immi.parquet'

In [33]:
fact_i94immi.write.mode("overwrite").parquet(parquet_outputs)

Py4JJavaError: An error occurred while calling o352.parquet.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:424)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:524)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:290)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:271)
	at org.apache.spark.sql.DataFrameWriter.save(Da